Import Packages

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import itertools

Files for Gephi

In [16]:
df = pd.read_csv(r'./dataset.csv')

df2 = df[["track_id","writers","track_name","track_pos"]]
df2_dict = df2.to_dict(orient='records')

d = []

for f in df2_dict:
    d.extend(f["writers"].split("-"))

set_d = set([a.strip() for a in d])
dict_vett_writers = []
support = []

for writer in set_d:
    for d in df2_dict:
        if writer in [a.strip() for a in d["writers"].split("-")]:
            support.append({"name":d["track_name"],"pos":d["track_pos"]})
    dict_vett_writers.append({"writer":writer,"tracks":support})
    support = []
list_edges = []
l_e_new = []
list_weights = []

for dic in dict_vett_writers:
    for dic2 in dict_vett_writers:
        for song in dic["tracks"]:
            if song in dic2["tracks"] and dic["writer"] != dic2["writer"]:
                if {"source":dic["writer"],"target":dic2["writer"]} in list_edges:
                    index = list_edges.index({"source":dic["writer"],"target":dic2["writer"]})
                    list_weights[index] += (10-song["pos"])
                else:
                    common_tracks =[]
                    list_edges.append({"source":dic["writer"],"target":dic2["writer"]})
                    for t1 in dic["tracks"]:
                        if t1 in dic2["tracks"]:
                            common_tracks.append(t1)
                    l_e_new.append({"source":dic["writer"],"target":dic2["writer"],"tracks":common_tracks})
                    list_weights.append(1 + (10-song["pos"]))

for weight in list_edges:
    a = weight["source"]
    b = weight["target"]
    index = list_edges.index(weight)
    try:
        list_edges.remove({"source":b,"target":a})
        list_weights.remove(list_weights[index])
    except:
        pass

lista =list(set([str(a["writer"]).strip() for a in dict_vett_writers]))
print(len(lista))
for w in lista:
    if w not in [b["source"] for b in list_edges]:
        lista.remove(w)


985


Creation of the files

In [3]:
writers = open("artisti.csv","w")
writers.write("Id\n")
writers.write('\n'.join(lista))
writers.close()

writersTracks = open("artisti_e_canzoni.csv","w")
writersTracks.write("Writer,Songs\n")
writersTracks.write('\n'.join([str(a["writer"]) + "," + str(a["tracks"]) for a in dict_vett_writers]))
writersTracks.close()

outfile = open("edges_list.csv","w")
outfile.write("Source,Target,Weight\n")
outfile.write('\n'.join([str(list_edges[i]["source"]) + "," + str(list_edges[i]["target"]) +","+ str(list_weights[i]) for i in range( len(list_edges))]))
outfile.close()

Create a Graph

In [49]:
G = nx.Graph()
G.add_nodes_from(lista)
ebunch = [(list_edges[i]["source"],list_edges[i]["target"],{"weight":list_weights[i]}) for i in range( len(list_edges))]
G.add_edges_from(ebunch)

#degree centrality
print("degree centrality")
print(nx.degree_centrality(G),'\n')

#degree distribution
print("degree distribution")
print(nx.degree_histogram(G),'\n') #find probability

#pagerank
print("pagerank")
print(nx.pagerank(G, weight="weight"),'\n')

#average clustering coefficient
print("average clustering coefficient")
print(nx.average_clustering(G,weight="weight"),'\n')



#k-cliques
cliqueMatrix = []
cliqueMatrix.append([])
cliqueMatrix.append([])
for k in range(2,8):
    i = 0
    cliqueMatrix.append([])
    for clique in nx.find_cliques(G):
        if len(clique) == k:
            i += 1
            cliqueMatrix[k].append(clique)
        elif len(clique) > k:
            i += len(list(itertools.combinations(clique, k)))
    print(k,"-clique: ", i)
print('\n')

#clique edges experimental
def travel_list(l,k):
    for el in l:
        a = el.get(k,None)
        if a == None:
            continue
        else:
            return a
        
def check_integrity(s,t):
    for el in s:
        pass

cliqueEdges= []
target = [{str(a["source"] + a["target"]):{"s":a["source"],"d":a["target"],"tracks":','.join([f["name"] + " - " + str(f["pos"]) for f in a["tracks"]])}} for a in l_e_new]
print("target - ", target)
for i in range(len(cliqueMatrix[6])):
    cliqueEdges.append([])
    for node1 in cliqueMatrix[6][i]:
        s = []
        for node2 in cliqueMatrix[6][i]:
            t = travel_list(target,str(node1 + node2))
            if not t in s and t is not None:
                s.append(t)
        cliqueEdges.append(list(s))

result = [c for c in cliqueEdges ]#if len(c)>1]
print("RES - ", result)
newfile = open("ce.txt","w")
newfile.write('\n'.join([str(b["s"]) + " | " + str(b["d"]) + " | " + str(b["tracks"]) for a in result for b in a])) #"," + str(' | '.join([str(b["name"]) +","+ str(b["pos"]) for b in a["tracks"]])) for a in result]))
newfile.close()
#############################

#closeness centrality
print("closeness centrality")
print(nx.closeness_centrality(G),'\n')

#betweenness centrality
print("betweenness centrality")
print(nx.betweenness_centrality(G,k=len(G.nodes()), normalized=True, weight="weight"),'\n')



#Degree Centrality, Degree Distibution, PageRank, Local Clustering Coefficient, N-cliques, Closeness, Betweenness

degree centrality
{'F. Zanotti': 0.00985761226725082, 'M. Vicino': 0.002190580503833516, 'N. Fasani': 0.004381161007667032, 'D. Ciaramella': 0.004381161007667032, 'M. Lusini': 0.001095290251916758, 'R. Rubino': 0.001095290251916758, 'R. Sanna': 0.001095290251916758, 'E. Croce': 0.001095290251916758, 'M. Tronco e G. Servillo': 0.004381161007667032, 'S. Endrigo': 0.002190580503833516, 'E. Ruggeri': 0.004381161007667032, 'G. Pegoraro': 0.001095290251916758, 'F. Poli': 0.001095290251916758, 'C. A. Rossi': 0.00547645125958379, 'M. Battistini': 0.004381161007667032, 'Nisa': 0.00547645125958379, 'S. Mattoccia': 0.002190580503833516, 'S. Dionigi': 0.00547645125958379, 'F. Minoia': 0.002190580503833516, 'P. Barabani': 0.0032858707557502738, 'P. Mammaro': 0.004381161007667032, 'A. Rizzo': 0.001095290251916758, 'P. Armenise': 0.001095290251916758, 'D. Sinigallia': 0.0032858707557502738, 'A. Bergamaschi': 0.001095290251916758, 'Irama': 0.0032858707557502738, 'P. Soffici': 0.001095290251916758, 'G.